<a href="https://colab.research.google.com/github/AndrewRucker/Fantasy-Football-and-Betting/blob/main/BestBall/Underdog/Rookie-and-Sophomores/Rookie_and_Soph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from IPython.core.pylabtools import figsize
from matplotlib import pyplot as plt
sns.set_context(font_scale=2)

# to bypass warnings in various dataframe assignments
pd.options.mode.chained_assignment = None

In [13]:
#Read in CSV to dataframe
url = 'https://raw.githubusercontent.com/AndrewRucker/Fantasy-Football-and-Betting/main/BestBall/Underdog/UD%20Teams.xlsx%20-%20UD%20Rookies%20%2B%20Sophomores.csv'
df = pd.read_csv(url, index_col='Team')

df

,POS,Name,TEAM,Round,Pick,ADP,Value,Draft Start,Cumulative Val
Team,,,,,,,,,
1,W/T,Jamar Chase,CIN,1,1.0,1.1,-0.1,4/13/2022,5.4
1,RB,Breece Hall,NaN,2,8.0,8.2,-0.2,4/13/2022,5.4
1,QB,Justin Fields,CHI,3,9.0,7.2,1.8,4/13/2022,5.4
1,QB,Mac Jones,NE,4,16.0,14.3,1.7,4/13/2022,5.4
1,RB,Elijah Mitchell,SF,5,17.0,17.5,-0.5,4/13/2022,5.4
...,...,...,...,...,...,...,...,...,...
77,W/T,George Pickens,NaN,8,31.0,29.9,1.1,4/19/2022,13.7
77,W/T,Jahan Dotson,NaN,9,34.0,33.3,0.7,4/19/2022,13.7
77,RB,Isaiah Spiller,NaN,10,39.0,34.8,4.2,4/19/2022,13.7


In [14]:
df1 = df.groupby('Team')['POS'].apply(lambda x: ', '.join(x.astype(str))).reset_index()

df1['RB'] = df1['POS'].str.count('RB')
df1['W/T'] = df1['POS'].str.count('W/T')
df1['QB'] = df1['POS'].str.count('QB')

df1['build'] = df1['QB'].astype(str) + "/" + df1['RB'].astype(str) + "/" + df1['W/T'].astype(str)
df1 = df1.drop(columns=['POS', 'RB', 'W/T', 'QB'])

In [15]:
df2 = pd.merge(df, df1, how='left', on = 'Team')
df3 = df2.groupby('Team')['Name'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
df2 = pd.merge(df2, df3, how='left', on='Team')

In [16]:
final_df = df2.drop(columns=['POS', 'Name_x', 'TEAM', 'Round', 'Pick', 'ADP', 'Value'])
final_df = final_df.drop_duplicates()
final_df ['Players'] = final_df['Name_y']
final_df = final_df.drop(columns=['Name_y'])
final_df

split_df = pd.concat([final_df['Team'], final_df['Players'].str.split(', ', expand=True)], axis=1)
split_df.rename({0 : 1, 1 : 2, 2 : 3, 3 : 4, 4 : 5, 5 : 6, 6 : 7, 7 : 8, 8 : 9, 9 : 10, 10 : 11, 11 : 12}, axis='columns', inplace=True)

In [36]:
#Exposure calc
exp_df = df2['Name_x'].value_counts()
exp_df = exp_df.reset_index()
exp_df['Count'] = exp_df['Name_x']
exp_df['%'] = round((exp_df['Count'] / df2['Team'].max()) * 100, 1)
exp_df = exp_df.drop(columns=['Name_x'])
exp_df

,index,Count,%
0,Rondale Moore,39,50.6
1,Trevor Lawrence,34,44.2
2,Nico Collins,31,40.3
3,Jahan Dotson,30,39.0
4,Kadarius Toney,29,37.7
...,...,...,...
58,Malik Willis,1,1.3
59,Justyn Ross,1,1.3
60,Wandale Robinson,1,1.3
61,Sam Howell,1,1.3


In [17]:
#Count of builds
build_count = final_df['build'].value_counts()
build_count

2/3/7    43
2/4/6    22
2/2/8     6
3/2/7     4
3/4/5     1
3/3/6     1
Name: build, dtype: int64

In [18]:
#Cumulative value description
cum_val_sum = final_df['Cumulative Val'].describe()
cum_val_sum

count    77.000000
mean      4.067532
std       8.384768
min     -15.800000
25%       0.000000
50%       4.100000
75%       8.200000
max      23.500000
Name: Cumulative Val, dtype: float64

In [19]:
#Function that returns all teams with player passed, along with the builds used with the player
def player_team_search(name):

      #Change for number of rounds
      rounds = 12
      i = 1

      with_player = final_df.loc[final_df['Players'].str.contains(name, case=False)]
      print("Structural builds with " + name + " on the roster")
      print(with_player['build'].value_counts())

      #Building out round by round breakdown for teams with given player
      split_df = pd.concat([final_df['Team'], final_df['Players'].str.split(', ', expand=True)], axis=1)
      split_df.rename({0 : 1, 1 : 2, 2 : 3, 3 : 4, 4 : 5, 5 : 6, 6 : 7, 7 : 8, 8 : 9, 9 : 10, 10 : 11, 11 : 12}, axis='columns', inplace=True)
      split_with_player = split_df[split_df.isin([name]).any(axis=1)]
      while i <= rounds:
        print("===============================================================")
        print("Round " + str(i) + " Selections with " + name + " on the team \n")
        print(split_with_player[i].value_counts())
        i=i+1

      #List of teams with given player on roster
      print("===========================================================")
      print("Teams with " + name + " on the roster \n")
      print(with_player['Players'].value_counts())


player_team_search("Najee Harris")

Structural builds with Najee Harris on the roster
2/3/7    15
2/4/6     2
2/2/8     2
3/2/7     2
Name: build, dtype: int64
Round 1 Selections with Najee Harris on the team 

Najee Harris    21
Name: 1, dtype: int64
Round 2 Selections with Najee Harris on the team 

Trevor Lawrence     10
Trey Lance           5
Justin Fields        3
Javonte Williams     1
Breece Hall          1
Jaylen Waddle        1
Name: 2, dtype: int64
Round 3 Selections with Najee Harris on the team 

Zach Wilson          6
Kyle Pitts           4
Justin Fields        3
Elijah Moore         2
Travis Etienne       1
Jaylen Waddle        1
Mac Jones            1
Devonta Smith        1
Trevor Lawrence      1
Amon-Ra St. Brown    1
Name: 3, dtype: int64
Round 4 Selections with Najee Harris on the team 

Elijah Moore         6
Devonta Smith        4
Mac Jones            4
Zach Wilson          2
Amon-Ra St. Brown    2
Travis Etienne       2
Justin Fields        1
Name: 4, dtype: int64
Round 5 Selections with Najee Harris